In the program, we have the last digit of sid `7`, so the order no. is `7`,$S1$ is 688 *China Overseas*, initial stock price `12.18`, volatility `43.6%`; $S2$ is 857 *Petrochina*, initial stock price `6.03`, volatility `30.0%`; $S1/S2$ correlation coefficient `0.304`, Group 3.

For Group 3, the $F\%=102.6\%, UB\%=130.0\%, A\%=122.0\%$

All codes below were executed on an Intel Core i5-12500H chip Windows platform with 40G of RAM. For the programs below, about 1.5G available RAM is required.

Programmes are Executed on Python 3.10, packages of `numpy` and `tqdm` are required.

### Q2

The payoff of an *Average Worst-of Put Option* with two stocks *S1* and *S2* is based on the following formula:  $\max (100\% – A, 0)$ payable at maturity (*t = T = 0.75* year from start date). 

where: 

1. $S_{1,0}, S_{2,0}$ = stock price at time $t=0$
2. $S_{1,1}, S_{2,1}$ = stock price at time $t=0.25$ year
3. $S_{1,2}, S_{2,2}$ = stock price at time $t=0.75$ year
4. $A=(B_1+B_2)/2$
5. $B_1=\min (S_{1,1}/S_{1,0}, S_{2,1}/S_{2,0})$
6. $B_2=\min (S_{1,2}/S_{1,0}, S_{2,2}/S_{2,0})$

Continuously compounded interest rate *r = 4.17% p.a*. 

Calculate the fair price of the option as of the start date (time *t=0*). 

Note: The answers should be a percentage (or a decimal number) smaller than 30%, and there is no need to multiply the answers with *S1* and/or *S2*. 

### Q2(i)

Use a Monte Carlo scheme with time steps *N = 150*, i.e. $\Delta t=T/N=1/200$ (refer to the discretization scheme in Topic 1-2, slides 37 and 38).  Give the answers with: (a) 10000 paths; (b) 300000 paths.  Record the computation times in each case. 

[Note: in this part, don’t use the exact discretization scheme.  Marks will be deducted if the exact scheme is adopted.] 

The following is a slow version following the slide 37 and 38 in Topic 1-2, step by step following a large table.

In [1]:
import numpy as np
import time
from tqdm import tqdm

# Parameters
S1_0 = 12.18  # Initial stock price of China Overseas
S2_0 = 6.03   # Initial stock price of Petrochina
sigma1 = 43.6 / 100  # Volatility of China Overseas
sigma2 = 30.0 / 100  # Volatility of Petrochina
rho = 0.304    # Correlation coefficient
r = 4.17 / 100     # Continuously compounded interest rate
N = 150        # Number of time steps
dt = 1 / 200     # Time step size
T = dt * N       # Maturity time
t1 = 0.25      # First observation time
t2 = 0.75      # Second observation time
num_paths_1 = 10_000    # Number of paths for case (a)
num_paths_2 = 300_000   # Number of paths for case (b)

# Generate correlated random numbers
def generate_correlated_randoms(time_step: int, rho: float) -> tuple:
    Z1 = np.random.normal(0, 1, time_step)
    Z2 = np.random.normal(0, 1, time_step)
    Z2_correlated = rho * Z1 + np.sqrt(1 - rho**2) * Z2
    return Z1, Z2_correlated

# Monte Carlo simulation
def monte_carlo_simulation(num_paths: int) -> float:
    S1 = np.zeros((num_paths, N, 3))
    S1[:, 0, 0] = S1_0
    S2 = np.zeros((num_paths, N, 3))
    S2[:, 0, 0] = S2_0
    payoffs = np.zeros(num_paths)
    
    for i in tqdm(range(num_paths)):
        Z1, Z2 = generate_correlated_randoms(N, rho)
        S1[i, :, 1] = Z1
        S2[i, :, 1] = Z2
        for j in range(N):
            S1[i, j, 2] = dt * r * S1[i, j, 0] + sigma1 * S1[i, j, 0] * S1[i, j, 1] * np.sqrt(dt) #delta S
            S2[i, j, 2] = dt * r * S2[i, j, 0] + sigma2 * S2[i, j, 0] * S2[i, j, 1] * np.sqrt(dt) #delta S
            if j != (N - 1): 
                S1[i, j + 1, 0] = S1[i, j, 0] + S1[i, j, 2]   
                S2[i, j + 1, 0] = S2[i, j, 0] + S2[i, j, 2]   
        
        S1_t1 = S1[i, int(t1/dt)-1, 0]
        S2_t1 = S2[i, int(t1/dt)-1, 0]
        S1_t2 = S1[i, int(t2/dt)-1, 0]
        S2_t2 = S2[i, int(t2/dt)-1, 0]
        
        B1 = np.minimum(S1_t1 / S1_0, S2_t1 / S2_0)
        B2 = np.minimum(S1_t2 / S1_0, S2_t2 / S2_0)
        A = (B1 + B2) / 2
        payoffs[i] = np.maximum(1.0 - A, 0)
    
    discounted_payoff = np.exp(-r * T) * payoffs
    fair_price = np.mean(discounted_payoff)
    return fair_price

# Case (a): 10,000 paths
print("Now calculating Case (a): ",end=" ")
start_time = time.time()
fair_price_1 = monte_carlo_simulation(num_paths_1)
computation_time_1 = time.time() - start_time
print(f"Fair price (10,000 paths): {fair_price_1 * 100:.2f}%")
print(f"Computation time (10,000 paths): {computation_time_1:.2f} seconds\n\n")

# Case (b): 300,000 paths
print("Now calculating Case (b): ",end=" ")
start_time = time.time()
fair_price_2 = monte_carlo_simulation(num_paths_2)
computation_time_2 = time.time() - start_time
print(f"Fair price (300,000 paths): {fair_price_2 * 100:.2f}%")
print(f"Computation time (300,000 paths): {computation_time_2:.2f} seconds")

Now calculating Case (a):  

100%|██████████| 10000/10000 [00:04<00:00, 2343.03it/s]


Fair price (10,000 paths): 13.14%
Computation time (10,000 paths): 4.29 seconds


Now calculating Case (b):  

100%|██████████| 300000/300000 [02:01<00:00, 2472.55it/s]


Fair price (300,000 paths): 13.07%
Computation time (300,000 paths): 121.64 seconds


The method below executes the same as above but have a faster calculation.

In [2]:
import numpy as np
import time

# Parameters
S1_0 = 12.18  # Initial stock price of China Overseas
S2_0 = 6.03   # Initial stock price of Petrochina
sigma1 = 43.6 / 100  # Volatility of China Overseas
sigma2 = 30.0 / 100  # Volatility of Petrochina
rho = 0.304    # Correlation coefficient
r = 4.17 / 100     # Continuously compounded interest rate
N = 150        # Number of time steps
dt = 1 / 200     # Time step size
T = dt * N       # Maturity time
t1 = 0.25      # First observation time
t2 = 0.75      # Second observation time
num_paths_1 = 10_000    # Number of paths for case (a)
num_paths_2 = 300_000   # Number of paths for case (b)

# Generate correlated random numbers
def generate_correlated_randoms(num_paths: int, time_step: int, rho: float) -> tuple:
    Z1 = np.random.normal(0, 1, (num_paths, time_step))
    Z2 = rho * Z1 + np.sqrt(1 - rho**2) * np.random.normal(0, 1, (num_paths, time_step))
    return Z1, Z2

# Monte Carlo simulation
def monte_carlo_simulation(num_paths: int) -> float:
    Z1, Z2 = generate_correlated_randoms(num_paths, N, rho)
    S1 = np.zeros((num_paths, N))
    S2 = np.zeros((num_paths, N))
    S1[:, 0] = S1_0
    S2[:, 0] = S2_0
    
    # Simulate price path
    for j in range(1, N):
        S1[:, j] = S1[:, j - 1] + dt * r * S1[:, j - 1] + sigma1 * S1[:, j - 1] * Z1[:, j - 1] * np.sqrt(dt)
        S2[:, j] = S2[:, j - 1] + dt * r * S2[:, j - 1] + sigma2 * S2[:, j - 1] * Z2[:, j - 1] * np.sqrt(dt)        
    
    S1_t1 = S1[:, int(t1/dt)-1]
    S2_t1 = S2[:, int(t1/dt)-1]
    S1_t2 = S1[:, int(t2/dt)-1]
    S2_t2 = S2[:, int(t2/dt)-1]
        
    B1 = np.minimum(S1_t1 / S1_0, S2_t1 / S2_0)
    B2 = np.minimum(S1_t2 / S1_0, S2_t2 / S2_0)
    A = (B1 + B2) / 2
    payoffs = np.maximum(1.0 - A, 0)
    
    discounted_payoff = np.exp(-r * T) * payoffs
    fair_price = np.mean(discounted_payoff)
    return fair_price

# Case (a): 10,000 paths
print("Now calculating Case (a): ",end=" ")
start_time = time.time()
fair_price_1 = monte_carlo_simulation(num_paths_1)
computation_time_1 = time.time() - start_time
print(f"Fair price (10,000 paths): {fair_price_1 * 100:.2f}%")
print(f"Computation time (10,000 paths): {computation_time_1:.2f} seconds\n")

# Case (b): 300,000 paths
print("Now calculating Case (b): ",end=" ")
start_time = time.time()
fair_price_2 = monte_carlo_simulation(num_paths_2)
computation_time_2 = time.time() - start_time
print(f"Fair price (300,000 paths): {fair_price_2 * 100:.2f}%")
print(f"Computation time (300,000 paths): {computation_time_2:.2f} seconds")

Now calculating Case (a):  Fair price (10,000 paths): 13.21%
Computation time (10,000 paths): 0.09 seconds

Now calculating Case (b):  Fair price (300,000 paths): 13.12%
Computation time (300,000 paths): 5.31 seconds


### Q2(ii)

Use a Monte Carlo scheme with two time steps N = 2, $\Delta t_1 = 0.25, \Delta t_2 = 0.5$ (refer to the discretization scheme in Topic 1-2, slides 39, 40, 42).  Give the answers with: (a) 10000 paths; (b) 300000 paths.  Record the computation times in each case. 

In [3]:
import numpy as np
import time

# Parameters
S1_0 = 12.18  # Initial stock price of China Overseas
S2_0 = 6.03   # Initial stock price of Petrochina
sigma1 = 43.6 / 100  # Volatility of China Overseas
sigma2 = 30.0 / 100  # Volatility of Petrochina
rho = 0.304    # Correlation coefficient
r = 4.17 / 100     # Continuously compounded interest rate
T = 0.75       # Maturity time (in years)
dt1 = 0.25     # First time step (t1 = 0.25 years)
dt2 = 0.50     # Second time step (t2 = 0.75 years)
num_paths_1 = 10000    # Number of paths for case (a)
num_paths_2 = 300000   # Number of paths for case (b)

# Generate correlated random numbers
def generate_correlated_randoms(num_paths: int, rho: float) -> tuple:
    Z1 = np.random.normal(0, 1, num_paths)
    Z2 = np.random.normal(0, 1, num_paths)
    Z2_correlated = rho * Z1 + np.sqrt(1 - rho**2) * Z2
    return Z1, Z2_correlated

# Monte Carlo simulation
def monte_carlo_simulation(num_paths: int) -> float:
    # Generate correlated random numbers for t1 and t2
    Z1_t1, Z2_t1 = generate_correlated_randoms(num_paths, rho)
    Z1_t2, Z2_t2 = generate_correlated_randoms(num_paths, rho)

    # Simulate stock prices at t1 and t2
    S1_t1 = S1_0 * np.exp((r - 0.5 * sigma1**2) * dt1 + sigma1 * np.sqrt(dt1) * Z1_t1)
    S2_t1 = S2_0 * np.exp((r - 0.5 * sigma2**2) * dt1 + sigma2 * np.sqrt(dt1) * Z2_t1)
    S1_t2 = S1_t1 * np.exp((r - 0.5 * sigma1**2) * dt2 + sigma1 * np.sqrt(dt2) * Z1_t2)
    S2_t2 = S2_t1 * np.exp((r - 0.5 * sigma2**2) * dt2 + sigma2 * np.sqrt(dt2) * Z2_t2)

    B1 = np.minimum(S1_t1 / S1_0, S2_t1 / S2_0)
    B2 = np.minimum(S1_t2 / S1_0, S2_t2 / S2_0)
    A = (B1 + B2) / 2
    payoff = np.maximum(1.0 - A, 0)
    # Discount the payoffs to present value
    discounted_payoff = np.exp(-r * T) * payoff
    # Compute the fair price as the average of discounted payoffs
    fair_price = np.mean(discounted_payoff)
    return fair_price

# Case (a): 10,000 paths
print("Now calculating Case (a): ",end=" ")
start_time = time.time()
fair_price_1 = monte_carlo_simulation(num_paths_1)
computation_time_1 = time.time() - start_time
print(f"Fair price (10,000 paths): {fair_price_1 * 100:.2f}%")
print(f"Computation time (10,000 paths): {computation_time_1:.2f} seconds")

# Case (b): 300,000 paths
print("Now calculating Case (b): ",end=" ")
start_time = time.time()
fair_price_2 = monte_carlo_simulation(num_paths_2)
computation_time_2 = time.time() - start_time
print(f"Fair price (300,000 paths): {fair_price_2 * 100:.2f}%")
print(f"Computation time (300,000 paths): {computation_time_2:.2f} seconds")

Now calculating Case (a):  Fair price (10,000 paths): 13.30%
Computation time (10,000 paths): 0.00 seconds
Now calculating Case (b):  Fair price (300,000 paths): 13.14%
Computation time (300,000 paths): 0.04 seconds
